In [ ]:
import re
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml
import pandas as pd
import matplotlib

from google.colab import files

#Web Scraping

Collecting text from web articles using BeautifulSoup tools


In [ ]:
pp_links=[]
#2021 - 2018 specific format
for i in range(2021,2017,-1):
  pp_links.append('https://www.morh.hr/kategorija/protupozarne-aktivnosti-'+str(i)+'/page/')

data_all=[]
base='https://www.morh.hr'
for url_base in pp_links:

  for page in range(13):#max 13 pages of reports per year
    WP=url_base+str(page)+'/'
    web_page = bs4.BeautifulSoup(requests.get(WP, headers={   "UserAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.183 Safari/537.36"}).text, "lxml")
    primary=web_page.find(id='primary')
    for newsi in primary.find_all('li'):
      try:
        meta1=newsi.select('.entry-preview__meta')
        date=newsi.select('.date')[0].text.strip()
        tt=newsi.select('.entry-preview__summary')[0].text.strip()
        title=newsi.select('.entry-preview__title')[0]
        link=title.find('a')
        post1={'datestr':date, 'title':link.text.strip(),'href':link['href'],'summary':tt }
        data_all.append(post1)
      except Exception as e:
        print(e)



In [ ]:
df=pd.DataFrame(data=data_all)
bodies=[]
for ull_a in df['href']:
  txt=""
  article_page = bs4.BeautifulSoup(requests.get(ull_a, headers={"UserAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.183 Safari/537.36"}).text, "lxml")
  for b in article_page.select('.entry__content'):
    txt=txt + b.text.strip()
  bodies.append(txt)
df['bodies']=bodies

## Regex template for gathering information
Collection of data on: number of fires, number of flights, hours of flights, tons of water, tons of cargo, liters of foam, liters of fuel

In [ ]:
templates =  {"number_of_fires": ".* u gašenju (?P<number_of_fires>([0-9]+)) požar.* .*",
             "number_of_flights": ".* (?P<number_of_flights>(([0-9]+\.[0-9]+)|[0-9]+)) let.*",
             "hours_of_flights": ".* (?P<hours_of_flights>(([0-9]+)|([0-9]+\.[0-9]+)|[0-9]+:[0-9]+:[0-9]+)|([0-9]+:[0-9]+)) sat.* nalet.* .*",
             "tons_of_water":".* (?P<tons_of_water>(([0-9]+\.[0-9]+)|[0-9]+)) ton. vod.*(\.*|(,* .*))",
             "tons_of_cargo": ".* (?P<tons_of_cargo>(([0-9]+\.[0-9]+)|[0-9]+)|([0-9]+,[0-9]+)) ton. teret.*(\.*|(,* .*))",
             "liters_of_foam": ".* (?P<liters_of_foam>(([0-9]+\.[0-9]+)|[0-9]+)|([0-9]+,[0-9]+)) lit.* pjenil.*(\.*|(,* .*))",
             "liters_of_fuel": ".* (?P<liters_of_fuel>(([0-9]+\.[0-9]+)|[0-9]+)|([0-9]+,[0-9]+)) lit.* goriv.*(\.*|(,* .*))"}


Planes_temp is another regex template used to find information about the type of aircrafts used for firefighting

In [ ]:
planes_temp = {"number_of_airtractors": "((.*\s)|(^))(?P<number_of_airtractors>(jedan|jednim|dva|tri|četiri|pet)) ((protupožar.* avion.*\s)|(avion.*\s))*((Airtractor.*)|(Air tractor.*)) .*",
             "number_of_canadairs":"((.*\s)|(^))(?P<number_of_canadairs>(jedan|jednim|dva|tri|četiri|pet)) ((protupožar.* avion.*\s)|(avion.*\s))*Canadair.* .*",
             "number_of_helicopters": "((.*\s)|(^))(?P<number_of_helicopters>(jedan|jednim|dva|tri|četiri|pet)) helikop.* Mi.* .*",
             "helicopters": ".* helikop.* (?P<helicopters>((Mi-8 MTV-1)|(Mi-8 MTV)|(Mi8 MTV)|(Mi8 MTV1)|(Mi-8 MTV1)|(Mi8 MTV-1)|(Mi-8)|(Mi8)))\.*,* .*"
}

The found data is stored in dictioneries data_dict and aircrafts

In [ ]:
data_dict =  {"date": [],
             "number_of_fires": [],
             "number_of_flights": [],
             "hours_of_flights": [],
             "tons_of_water": [],
             "tons_of_cargo": [],
             "liters_of_foam": [],
             "liters_of_fuel": []
}
aircrafts = { "date": [],
             "text": [],
             "number_of_airtractors": [],
             "number_of_canadairs": [],
             "number_of_helicopters": [] }

## Function count_aircrafts
Searching for data on aircrafts and counting the number of the same type of planes used in specific fire

In [ ]:
def count_aircrafts(input_text, i):
  split_text = [s.strip() for s in re.split('[,.]', input_text)]
  split_text = list(filter(None, split_text))

  aircrafts['number_of_airtractors'].append(0)
  aircrafts['number_of_canadairs'].append(0)
  aircrafts['number_of_helicopters'].append(0)
  for string in split_text:

    for template_key, template in planes_temp.items():
      regex_temp = re.sub(r'\{\s*\d+\s*\}','.+',template) + '$'
      z = re.match(regex_temp, string, flags=re.I)

      if z:
          if template_key == "number_of_airtractors":
            if(re.match(r'(jedan)|(jednim)', z.groupdict()['number_of_airtractors'], flags=re.I)):
              aircrafts["number_of_airtractors"][i] += 1
            elif (re.match(r'dva', z.groupdict()['number_of_airtractors'], flags=re.I)):
              aircrafts["number_of_airtractors"][i] += 2
            elif (re.match(r'tri', z.groupdict()['number_of_airtractors'], flags=re.I)):
              aircrafts["number_of_airtractors"][i] += 3
            elif (re.match(r'četiri', z.groupdict()['number_of_airtractors'], flags=re.I)):
              aircrafts["number_of_airtractors"][i] += 4

          elif template_key == "number_of_canadairs":
            if (re.match(r'(jedan)|(jednim)', z.groupdict()['number_of_canadairs'], flags=re.I)):
                aircrafts["number_of_canadairs"][i] += 1
            elif (re.match(r'dva', z.groupdict()['number_of_canadairs'], flags=re.I)):
              aircrafts["number_of_canadairs"][i] += 2
            elif (re.match(r'tri', z.groupdict()['number_of_canadairs'], flags=re.I)):
              aircrafts["number_of_canadairs"][i] += 3
            elif (re.match(r'četiri', z.groupdict()['number_of_canadairs'], flags=re.I)):
              aircrafts["number_of_canadairs"][i] += 4
            elif (re.match(r'pet', z.groupdict()['number_of_canadairs'], flags=re.I)):
              aircrafts["number_of_canadairs"][i] += 5

          elif template_key == "number_of_helicopters":
            if(re.match(r'(jedan)|(jednim)', z.groupdict()['number_of_helicopters'], flags=re.I)):
              aircrafts["number_of_helicopters"][i] += 1
            elif (re.match(r'Dva', z.groupdict()['number_of_helicopters'], flags=re.I)):
              aircrafts["number_of_helicopters"][i] += 2
            elif (re.match(r'tri', z.groupdict()['number_of_helicopters'], flags=re.I)):
              aircrafts["number_of_helicopters"][i] += 3
            elif (re.match(r'četiri', z.groupdict()['number_of_helicopters'], flags=re.I)):
              aircrafts["number_of_helicopters"][i] += 4
            elif template_key == "number_of_helicopters":
              aircrafts["number_of_helicopters"][i] += 5

      else:
          if template_key == "number_of_airtractors":
            aircrafts["number_of_airtractors"][i] += 0
          elif template_key == "number_of_canadairs":
            aircrafts["number_of_canadairs"][i] += 0
          elif template_key == "number_of_helicopters":
            aircrafts["number_of_helicopters"][i] += 0




In [ ]:
i = 0
for bodie in df.bodies:
    aircrafts["date"] = df['datestr']
    aircrafts["text"] = df['bodies']
    count_aircrafts(bodie, i)
    i += 1

df2 = pd.DataFrame(aircrafts)

## Function find_template
Searching for data defined in template

In [ ]:
# info about latest fire
def find_template(input_text, i):
  for template_key, template in templates.items():
    regex_temp = re.sub(r'\{\s*\d+\s*\}','.+',template) + '$'
    z = re.match(regex_temp, input_text, flags=re.I | re.MULTILINE | re.DOTALL)
    regex_temp = re.sub(r'\{\s*\d+\s*\}','.+',template) + '$'


    if z:
      if template_key == "number_of_fires":
        data_dict["number_of_fires"].append(z.groupdict()['number_of_fires'])
      elif template_key == "number_of_flights":
        data_dict["number_of_flights"].append(z.groupdict()['number_of_flights'])
      elif template_key == "hours_of_flights":
        data_dict["hours_of_flights"].append(z.groupdict()['hours_of_flights'])
      elif template_key == "tons_of_water":
        data_dict["tons_of_water"].append(z.groupdict()['tons_of_water'])
      elif template_key == "tons_of_cargo":
        data_dict["tons_of_cargo"].append(z.groupdict()['tons_of_cargo'])
      elif template_key == "liters_of_foam":
        data_dict["liters_of_foam"].append(z.groupdict()['liters_of_foam'])
      elif template_key == "liters_of_fuel":
        data_dict["liters_of_fuel"].append(z.groupdict()['liters_of_fuel'])

    else:
      if template_key == "number_of_fires":
        data_dict["number_of_fires"].append("")
      elif template_key == "number_of_flights":
        data_dict["number_of_flights"].append("")
      elif template_key == "hours_of_flights":
        data_dict["hours_of_flights"].append("")
      elif template_key == "tons_of_water":
        data_dict["tons_of_water"].append("")
      elif template_key == "tons_of_cargo":
        data_dict["tons_of_cargo"].append("")
      elif template_key == "liters_of_foam":
        data_dict["liters_of_foam"].append("")
      elif template_key == "liters_of_fuel":
        data_dict["liters_of_fuel"].append("")



In [ ]:
i = 0
j = 0
for j in range(len(df.bodies)):
    data_dict["date"] = df['datestr']
    find_template(df['bodies'][j], i)
    i += 1
    j += 1

print(data_dict)
df_data = pd.DataFrame(data_dict)


Combine two data frames into one and export to an excel table

In [ ]:
df_data.drop('date', inplace=True, axis = 1)
complete = pd.concat([df2, df_data], axis="columns")


In [ ]:
complete.to_excel("final2021-18.xlsx")
files.download('final2021-18.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>